### Student Information
Name: 蕭泓佐

Student ID: r13922163

GitHub ID: davidshiao55

Kaggle name: David Shiao

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

## Environment setup

In [3]:
%pip install emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 582.2 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=6bae018d564ae20840e34baf1d9a8414c1241649f22f9f4364a8230949fd2bbf
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.14.0
    Uninstalling emoji-2.14.0:
      Successfully uninstalled emoji-2.14.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
input_path = 'data'
save_dir = './models/Bertweet-base_v3'
model_path = "./models/Bertweet-base_v3/ep_3"

os.makedirs(save_dir, exist_ok=True)

# input_path = '/kaggle/input/dm-2024-isa-5810-lab-2-homework'
# model_path = '/kaggle/input/bertweet/transformers/default/4/ep_5'

In [21]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Hyperparameters

In [ ]:
model_name = "vinai/bertweet-base"
num_epochs = 3
batch_size = 128
max_length = 96
lr = 2e-5
step_size = 2
gamma = 0.8

## Data

In [5]:
# !mkdir -p ~/.kaggle
# !cp /DM2024-Lab2-Homework/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c dm-2024-isa-5810-lab-2-homework
# !mkdir -p ./data
# !unzip -q dm-2024-isa-5810-lab-2-homework.zip -d ./data
# !rm dm-2024-isa-5810-lab-2-homework.zip

In [23]:
import pandas as pd

data_ids = pd.read_csv(f'{input_path}/data_identification.csv')
data_ids.head()

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [24]:
train_data_ids = data_ids[data_ids['identification'] == 'train'].drop(['identification'], axis=1)
test_data_ids = data_ids[data_ids['identification'] == 'test'].drop(['identification'], axis=1)

train_data_ids.head()

,tweet_id
1,0x29e452
2,0x2b3819
4,0x2a2acc
5,0x2a8830
6,0x20b21d


In [25]:
emotion_labels = pd.read_csv(f'{input_path}/emotion.csv')
emotion_labels.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [26]:
tweets_df = pd.read_json(f'{input_path}/tweets_DM.json', lines=True)

# Use json_normalize to flatten only the `_source` column and keep other columns intact
source_df = pd.json_normalize(tweets_df['_source'])

# Concatenate the flattened `_source` columns with the original dataframe
tweets_df = pd.concat([tweets_df.drop(columns=['_source']), source_df], axis=1)

tweets_df.head()

,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [27]:
train_data = pd.merge(train_data_ids, emotion_labels, on='tweet_id', how='inner')
train_data = pd.merge(train_data, tweets_df, left_on='tweet_id', right_on='tweet.tweet_id', how='inner')
train_data.head()

,tweet_id,emotion,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,0x29e452,joy,809,hashtag_tweets,2015-01-17 03:07:03,tweets,[],0x29e452,Huge Respect🖒 @JohnnyVegasReal talking about l...
1,0x2b3819,joy,808,hashtag_tweets,2016-07-02 09:34:06,tweets,"[spateradio, app]",0x2b3819,Yoooo we hit all our monthly goals with the ne...
2,0x2a2acc,trust,16,hashtag_tweets,2016-08-15 18:18:39,tweets,[],0x2a2acc,@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...
3,0x2a8830,joy,768,hashtag_tweets,2017-02-11 08:49:46,tweets,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",0x2a8830,Come join @ambushman27 on #PUBG while he striv...
4,0x20b21d,anticipation,70,hashtag_tweets,2016-11-23 05:37:10,tweets,"[strength, bones, God]",0x20b21d,@fanshixieen2014 Blessings!My #strength little...


In [28]:
test_data = pd.merge(test_data_ids, tweets_df, left_on='tweet_id', right_on='tweet.tweet_id', how='inner')
test_data.head()

,tweet_id,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,0x28cc61,107,hashtag_tweets,2017-01-17 14:13:32,tweets,[],0x28cc61,@Habbo I've seen two separate colours of the e...
1,0x2db41f,728,hashtag_tweets,2015-10-17 06:46:20,tweets,[],0x2db41f,@FoxNews @KellyannePolls No serious self respe...
2,0x2466f6,491,hashtag_tweets,2016-12-19 03:50:27,tweets,[womendrivers],0x2466f6,"Looking for a new car, and it says 1 lady owne..."
3,0x23f9e9,28,hashtag_tweets,2017-04-09 19:32:19,tweets,[robbingmembers],0x23f9e9,@cineworld “only the brave” just out and fount...
4,0x1fb4e1,925,hashtag_tweets,2016-01-15 11:59:31,tweets,[],0x1fb4e1,Felt like total dog 💩 going into open gym and ...


## Data Exploration

In [29]:
# analyze class distribution
train_data['emotion'].value_counts()

emotion
joy             516017
anticipation    248935
trust           205478
sadness         193437
disgust         139101
fear             63999
surprise         48729
anger            39867
Name: count, dtype: int64

## Data preprocessing

In [30]:
from emoji import demojize
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())

In [31]:
def preprocess_text(text):
    # Remove <LH> tags
    text = text.replace('<LH>', '')
    
    # normalize tweet
    text = normalizeTweet(text)
    
    return text

In [32]:
# Define the mapping dictionary
emotion_mapping = {
    "joy": 0,
    "anticipation": 1,
    "trust": 2,
    "sadness": 3,
    "disgust": 4,
    "fear": 5,
    "surprise": 6,
    "anger": 7,
}

# Reverse mapping for decoding
reverse_emotion_mapping = {v: k for k, v in emotion_mapping.items()}


In [33]:
# Apply preprocessing function to the text column
train_data['processed_text'] = train_data['tweet.text'].apply(preprocess_text)
train_data['emotion_label'] = train_data["emotion"].map(emotion_mapping)

In [34]:
from sklearn.model_selection import train_test_split

X = train_data['processed_text']
y = train_data['emotion_label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, y_train = X, y

## Model training

In [35]:
import torch
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim.lr_scheduler import StepLR
from torch.optim import AdamW

In [36]:
class TweetDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_length=128):
        self.texts = texts
        self.labels = labels  # Labels can be None for inference
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        item = {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
        }
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
class_counts = np.bincount(y_train.values)
class_weights = 1.0 / class_counts
sample_weights = np.array([class_weights[label] for label in y_train.values])

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

In [39]:
# Initialize datasets
train_dataset = TweetDataset(X_train.values, y_train.values, tokenizer, max_length=max_length)
val_dataset = TweetDataset(X_val.values, y_val.values, tokenizer, max_length=max_length)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size * 2)

In [40]:
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
from sklearn.metrics import f1_score

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    model.train()
    total_train_loss = 0

    # Training phase
    for batch in tqdm(train_dataloader, desc="Training"):
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        
        # Backward pass
        loss.backward()
        
        # Clip the gradient to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)        
        
        # Update parameters
        optimizer.step()

    # Step the scheduler after each epoch
    scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validating"):
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)
            
            # Forward pass for validation
            outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            
            total_val_loss += loss.item()
            
            # Store predictions and true labels
            _, preds = torch.max(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(b_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_dataloader)
    
    # Calculate mean F1 score
    mean_f1 = f1_score(all_labels, all_preds, average='macro')
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}")
    print(f"Epoch {epoch + 1}/{num_epochs}, Mean F1 Score: {mean_f1:.4f}")

    # Save model checkpoint
    save_path = f"{save_dir}/ep_{epoch + 1}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Model and Tokenizer checkpoint saved to {save_path}")

## INFERENCE

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

In [17]:
test_data['processed_text'] = test_data['tweet.text'].apply(preprocess_text)
X_test = test_data['processed_text']

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

test_dataset = TweetDataset(X_test, labels=None, tokenizer=tokenizer, max_length=max_length)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size * 2)

2024-12-03 19:30:05.247538: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 19:30:05.289078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Set the model to evaluation mode
model.eval()

# Perform predictions
predictions = []
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)

        # Get logits from the model
        outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask)
        logits = outputs.logits

        # Get the predicted labels
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())

# Add predictions to the test DataFrame
test_data['emotion_label'] = predictions
test_data['emotion'] = test_data['emotion_label'].map(reverse_emotion_mapping)

# Keep only the desired columns
submission = test_data[['tweet_id', 'emotion']]

# Rename the 'tweet_id' column to 'id'
submission = submission.rename(columns={'tweet_id': 'id'})

# Save or display the predictions
submission.to_csv("submission.csv", index=False)
submission.head()

Testing: 100%|██████████| 1610/1610 [26:43<00:00,  1.00it/s]


,id,emotion
0,0x28cc61,sadness
1,0x2db41f,sadness
2,0x2466f6,sadness
3,0x23f9e9,sadness
4,0x1fb4e1,joy


In [19]:
test_data[['tweet.text', 'emotion']][:20]

,tweet.text,emotion
0,@Habbo I've seen two separate colours of the e...,sadness
1,@FoxNews @KellyannePolls No serious self respe...,sadness
2,"Looking for a new car, and it says 1 lady owne...",sadness
3,@cineworld “only the brave” just out and fount...,sadness
4,Felt like total dog 💩 going into open gym and ...,joy
5,#RYSENSHYNE @nydjlive plz ply me ne woa @man...,joy
6,"I am thankful for my husband, family, furry b...",trust
7,I have dreams but then there are dreams that f...,disgust
8,Fuck you @JoelOsteen God will judge you for yo...,disgust
9,Pretty sure he was safe...just because the thr...,disgust


## Analyze Data

### Analyze the proper sequence length for the task

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize without truncation
# token_lengths = [len(tokenizer.encode(text, truncation=False)) for text in train_data['processed_text']]
token_lengths = [len(tokenizer.encode(text, truncation=False)) for text in test_data['processed_text']]

print(f"Max token length: {max(token_lengths)}")
print(f"95th percentile token length: {np.percentile(token_lengths, 95)}")
print(f"Average token length: {np.mean(token_lengths)}")

Max token length: 385
95th percentile token length: 65.0
Average token length: 38.15450079131592


### Analyze for unrecognized tokens in the train data

In [ ]:
from transformers import AutoTokenizer
import json

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Access the tokenizer vocabulary
vocab = tokenizer.get_vocab()

# Identify unknown tokens in training data
unknown_tokens_analysis = []

# for text in train_data['processed_text']:
for text in test_data['processed_text']:
    tokens = tokenizer.tokenize(text)  # Tokenize the text
    unk_tokens = [token for token in tokens if token not in vocab]
    if unk_tokens:
        unknown_tokens_analysis.append({
            "text": text,
            "tokenized": tokens,
            "unknown_tokens": unk_tokens
        })

# Save results to a file
output_file = "unknown_tokens.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(unknown_tokens_analysis, f, indent=4, ensure_ascii=False)

print(f"Unknown tokens and tokenized results have been saved to {output_file}.")

Unknown tokens and tokenized results have been saved to unknown_tokens.json.


### Case study

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
i = 14
train_data['tweet.text'][i]

'#Congratulations to all our #Graduates from @StOswaldsPriLiv   From <LH> #learningisfun <LH> #primary #Liverpool #aspire <LH>'

In [ ]:
train_data['processed_text'][i]


'#Congratulations to all our #Graduates from @USER From #learningisfun #primary #Liverpool #aspire'

In [ ]:
tokenizer.tokenize(train_data['processed_text'][i])

['#@@',
 'Congratulations',
 'to',
 'all',
 'our',
 '#Gra@@',
 'du@@',
 'ates',
 'from',
 '@USER',
 'From',
 '#lear@@',
 'ning@@',
 'is@@',
 'fun',
 '#@@',
 'primary',
 '#Liverpool',
 '#@@',
 'aspire']